# RandomForest Model

In [15]:
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score,f1_score, confusion_matrix
from sklearn.model_selection import KFold,cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score


## Loading data

In [6]:
airplanedf = pd.read_csv('../AirplaneDataset/train.csv', index_col=0)
airplanedf.head()

,id,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,...,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,satisfaction
0,70172,Male,Loyal Customer,13,Personal Travel,Eco Plus,460,3,4,3,...,5,4,3,4,4,5,5,25,18.0,neutral or dissatisfied
1,5047,Male,disloyal Customer,25,Business travel,Business,235,3,2,3,...,1,1,5,3,1,4,1,1,6.0,neutral or dissatisfied
2,110028,Female,Loyal Customer,26,Business travel,Business,1142,2,2,2,...,5,4,3,4,4,4,5,0,0.0,satisfied
3,24026,Female,Loyal Customer,25,Business travel,Business,562,2,5,5,...,2,2,5,3,1,4,2,11,9.0,neutral or dissatisfied
4,119299,Male,Loyal Customer,61,Business travel,Business,214,3,3,3,...,3,3,4,4,3,3,3,0,0.0,satisfied


In [7]:
airplanedf.set_index('id', inplace=True)
def preprocessing(data, mode):
    categorical_indexes = [0, 1, 3, 4] + list(range(6, 20))
    if mode == 'train':
        categorical_indexes = categorical_indexes + [22]
        
    data.iloc[:,categorical_indexes] = data.iloc[:,categorical_indexes].astype('category')
    return data

airplanedf = preprocessing(airplanedf, 'train')

C:\Users\cyber\AppData\Local\Temp\ipykernel_33424\1100536982.py:7: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  data.iloc[:,categorical_indexes] = data.iloc[:,categorical_indexes].astype('category')


In [22]:
intdf = pd.DataFrame(airplanedf)
categorical_indexes = [0, 1, 3, 4, 22] + list(range(6, 20))
for i in categorical_indexes:
    intdf.iloc[:, i] = intdf.iloc[:, i].cat.codes
intdf.iloc[:,categorical_indexes] = intdf.iloc[:,categorical_indexes].astype('int')

intdf.head()

C:\Users\cyber\AppData\Local\Temp\ipykernel_33424\1276975896.py:4: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  intdf.iloc[:, i] = intdf.iloc[:, i].cat.codes


,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,Gate location,...,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,satisfaction
id,,,,,,,,,,,,,,,,,,,,,
70172,1,0,13,1,2,460,3,4,3,1,...,5,4,3,3,4,5,5,25,18.0,0
5047,1,1,25,0,0,235,3,2,3,3,...,1,1,5,2,1,4,1,1,6.0,0
110028,0,0,26,0,0,1142,2,2,2,2,...,5,4,3,3,4,4,5,0,0.0,1
24026,0,0,25,0,0,562,2,5,5,5,...,2,2,5,2,1,4,2,11,9.0,0
119299,1,0,61,0,0,214,3,3,3,3,...,3,3,4,3,3,3,3,0,0.0,1


In [30]:
#Check for null values
print(intdf.isnull().sum())

Gender                                 0
Customer Type                          0
Age                                    0
Type of Travel                         0
Class                                  0
Flight Distance                        0
Inflight wifi service                  0
Departure/Arrival time convenient      0
Ease of Online booking                 0
Gate location                          0
Food and drink                         0
Online boarding                        0
Seat comfort                           0
Inflight entertainment                 0
On-board service                       0
Leg room service                       0
Baggage handling                       0
Checkin service                        0
Inflight service                       0
Cleanliness                            0
Departure Delay in Minutes             0
Arrival Delay in Minutes             310
satisfaction                           0
dtype: int64


In [32]:
intdf.iloc[:, 21].fillna(intdf.iloc[:, 21].mean(), inplace=True)

In [33]:
#Check for null values
print(intdf.isnull().sum())

Gender                               0
Customer Type                        0
Age                                  0
Type of Travel                       0
Class                                0
Flight Distance                      0
Inflight wifi service                0
Departure/Arrival time convenient    0
Ease of Online booking               0
Gate location                        0
Food and drink                       0
Online boarding                      0
Seat comfort                         0
Inflight entertainment               0
On-board service                     0
Leg room service                     0
Baggage handling                     0
Checkin service                      0
Inflight service                     0
Cleanliness                          0
Departure Delay in Minutes           0
Arrival Delay in Minutes             0
satisfaction                         0
dtype: int64


In [35]:
X = intdf.iloc[:, 0:22]
y = intdf.iloc[:, 22]

In [36]:
X

,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,Gate location,...,Seat comfort,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes
id,,,,,,,,,,,,,,,,,,,,,
70172,1,0,13,1,2,460,3,4,3,1,...,5,5,4,3,3,4,5,5,25,18.0
5047,1,1,25,0,0,235,3,2,3,3,...,1,1,1,5,2,1,4,1,1,6.0
110028,0,0,26,0,0,1142,2,2,2,2,...,5,5,4,3,3,4,4,5,0,0.0
24026,0,0,25,0,0,562,2,5,5,5,...,2,2,2,5,2,1,4,2,11,9.0
119299,1,0,61,0,0,214,3,3,3,3,...,5,3,3,4,3,3,3,3,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94171,0,1,23,0,1,192,2,1,2,3,...,2,2,3,1,3,2,3,2,3,0.0
73097,1,0,49,0,0,2347,4,4,4,4,...,5,5,5,5,4,5,5,4,0,0.0
68825,1,1,30,0,0,1995,1,1,1,3,...,5,4,3,2,3,5,5,4,7,14.0


In [37]:
y

id
70172     0
5047      0
110028    1
24026     0
119299    1
         ..
94171     0
73097     1
68825     0
54173     0
62567     0
Name: satisfaction, Length: 103904, dtype: int32

In [38]:
X_train,X_test,Y_train,Y_test=train_test_split(X,y,test_size=0.3,random_state=123)

In [27]:
print("X_train shape: ")
print(X_train.shape)
print("X_test shape: ")
print(X_test.shape)
print("--------------")
print("Y_train shape: ")
print(Y_train.shape)
print("Y_test shape: ")
print(Y_test.shape)

X_train shape: 
(72732, 22)
X_test shape: 
(31172, 22)
--------------
Y_train shape: 
(72732,)
Y_test shape: 
(31172,)


## Default model (No hyperparameters applied)

In [39]:
model = RandomForestClassifier()
clf = model.fit(X_train, Y_train)
y_pred = clf.predict(X_test)
y_pred

array([1, 0, 0, ..., 0, 0, 0])

In [40]:
print('Accuracy: %.2f' % accuracy_score(Y_test, y_pred))

Accuracy: 0.96


In [41]:
kfold=KFold(n_splits=5)
cv_score=[]

x=X.values
y=y.values
model=RandomForestClassifier()

n_iter=0
for train_index, test_index in kfold.split(x):
    x_train, x_test=x[train_index],x[test_index]
    y_train, y_test=y[train_index],y[test_index]
    
    model.fit(x_train,y_train)
    y_pred_train=model.predict(x_train)
    y_pred_test=model.predict(x_test)
    
    n_iter=n_iter+1
    
    acc_train=f1_score(y_train,y_pred_train)
    acc_test=f1_score(y_test,y_pred_test)
    cv_score.append(acc_test)
    
    print("Modeling ",n_iter,"st")
    print('train set f1_score: ',acc_train)
    print('test set f1_score: ',acc_test)
    print("-----------------------------------------")
    
print('Mean test set f1_score: ',np.mean(cv_score))

Modeling  1 st
train set f1_score:  1.0
test set f1_score:  0.9566433566433566
-----------------------------------------
Modeling  2 st
train set f1_score:  1.0
test set f1_score:  0.9548387096774195
-----------------------------------------
Modeling  3 st
train set f1_score:  1.0
test set f1_score:  0.9536999942768843
-----------------------------------------
Modeling  4 st
train set f1_score:  1.0
test set f1_score:  0.9574551790029787
-----------------------------------------
Modeling  5 st
train set f1_score:  1.0
test set f1_score:  0.9563040509786072
-----------------------------------------
Mean test set f1_score:  0.9557882581158493


In [42]:
model=RandomForestClassifier()
param={'max_depth':[None,5,8],'class_weight':[None,'balanced'],'n_estimators':[100,200,300],'min_samples_split':[2,3,4]}

grid_model=GridSearchCV(model,param_grid=param, cv=5,scoring='f1')
grid_model.fit(X_train,Y_train)
result2=pd.DataFrame(grid_model.cv_results_)

In [43]:
grid_model.best_params_

{'class_weight': None,
 'max_depth': None,
 'min_samples_split': 2,
 'n_estimators': 200}

In [44]:
model = RandomForestClassifier(class_weight=None, max_depth=None, min_samples_split=2, n_estimators=200)
clf = model.fit(X_train, Y_train)
y_pred = clf.predict(X_test)
y_pred

array([1, 0, 0, ..., 0, 0, 0])

In [45]:
print('Accuracy: %.2f' % accuracy_score(Y_test, y_pred))

Accuracy: 0.96
